In [19]:
# 首先在服务器上clone当前仓库
!ssh riscv-server sudo rm -rf application_benchmark
!ssh riscv-server git clone https://github.com/LiuZhetan/application_benchmark.git

Cloning into 'application_benchmark'...
^C


In [20]:
# 然后进入仓库目录并执行测试脚本
# 该脚本会在服务器上执行redis的测试，并将测试结果保存在redis/results/目录下
!ssh riscv-server 'cd application_benchmark && sudo -S bash ./redis/run_benchmark.sh -p riscv
# 从远程服务器中下载redis/results/目录下的测试结果

Redis is running
Measuring latency:
Measuring intrinsic latency, it takes 80 seconds......
Max latency so far: 14 microseconds.
Max latency so far: 22 microseconds.
Max latency so far: 24 microseconds.
Max latency so far: 63 microseconds.
Max latency so far: 67 microseconds.
Max latency so far: 74 microseconds.
Max latency so far: 89 microseconds.

447275874 total runs (avg latency: 0.1789 microseconds / 178.86 nanoseconds per run).
Worst run took 498x longer than the average latency.
......Running benchmark......
1
"PING_BULK","230545.45"
"SET","142857.14"
"GET","181818.19"
"INCR","182400.00"
"LPUSH","138301.38"
"RPUSH","154830.78"
"LPOP","133333.33"
"RPOP","156250.00"
"SADD","183563.64"
"HSET","143771.42"
"SPOP","219130.44"
"ZADD","147058.81"
"ZPOPMIN","210333.33"
"LPUSH (needed to benchmark LRANGE)","141070.42"
"LRANGE_100 (first 100 elements)","19806.58"
"LRANGE_300 (first 300 elements)","5680.59"
"LRANGE_500 (first 450 elements)","3645.86"
"LRANGE_600 (first 600 elements)","2588.2

In [24]:
# 在本地测试脚本
!echo xxxxx | sudo -S bash run_benchmark.sh

[sudo] password for ubuntu_wsl: Redis is not running, starting Redis
Measuring latency:
0 1 0.16 453

......Running benchmark......
Clinet and query num X1
redis-benchmark -P 16 -d 32 -c 50 -n 10000 --csv 
Error from server: ERR unknown command '', with args beginning with: 
"test","rps","avg_latency_ms","min_latency_ms","p50_latency_ms","p95_latency_ms","p99_latency_ms","max_latency_ms"
Clinet and query num X2
redis-benchmark -P 16 -d 32 -c 100 -n 20000 --csv 
Error from server: ERR unknown command '', with args beginning with: 
"test","rps","avg_latency_ms","min_latency_ms","p50_latency_ms","p95_latency_ms","p99_latency_ms","max_latency_ms"
Clinet and query num X4
redis-benchmark -P 16 -d 32 -c 200 -n 40000 --csv 
Error from server: ERR unknown command '', with args beginning with: 
"test","rps","avg_latency_ms","min_latency_ms","p50_latency_ms","p95_latency_ms","p99_latency_ms","max_latency_ms"
Clinet and query num X8
redis-benchmark -P 16 -d 32 -c 400 -n 80000 --csv 
Error from ser